In [1]:
# Imports
import os
import numpy as np
import pandas as pd
import nibabel as nb
import warnings
warnings.filterwarnings('ignore')

# Figure imports
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px
import plotly.express as px
from plot_utils import plotly_template

# stats import
from scipy.stats import permutation_test
def statistic(condA, condB, axis):
    return np.nanmean(condA, axis=axis) - np.nanmean(condB, axis=axis)

# Define parameters
subjects = ['sub-001', 'sub-002', 'sub-003', 'sub-004',
            'sub-005', 'sub-006', 'sub-007', 'sub-008']
subjects_plot = ['sub-001', 'sub-002', 'sub-003', 'sub-004',
                 'sub-005', 'sub-006', 'sub-007', 'sub-008', 'group']
tasks = ['FullScreen', 'FullScreenAttendFix', 'FullScreenAttendBar']
rois = ['V1', 'V2', 'V3', 'V3AB', 'hMT+', 'LO',
        'VO', 'iIPS', 'sIPS', 'iPCS', 'sPCS', 'mPCS']

# Define folders
base_dir = '/home/mszinte/disks/meso_S/data/gaze_prf'
bids_dir = "{}".format(base_dir)
pp_dir = "{}/derivatives/pp_data".format(base_dir)

# analysis settings
best_voxels_num = 250
type_analyses = ['','_best{}'.format(best_voxels_num)]
n_permutation = 10000
cortical_mask = 'cortical'
n_ecc_bins=10

### Compute TSV files for fullscreen atention R2 comparison

In [2]:
# Create TSV files
group_tsv_dir = '{}/{}/prf/tsv'.format(pp_dir, 'group')
try: os.makedirs(group_tsv_dir)
except: pass

for task in tasks:
    for subject_num, subject in enumerate(subjects):
        # define folders
        fit_dir = '{}/{}/prf/fit'.format(pp_dir, subject)
        mask_dir = '{}/{}/masks'.format(pp_dir, subject)
        tsv_dir = '{}/{}/prf/tsv'.format(pp_dir, subject)
        try: os.makedirs(tsv_dir)
        except: pass

        # load pRF threshold masks
        th_mat = nb.load('{}/{}_task-{}_prf_threshold.nii.gz'.format(mask_dir,subject,task)).get_fdata()

        # load fit parameters x by threshold
        r2_th_mat = nb.load('{}/{}_task-{}_par-r2.nii.gz'.format(fit_dir,subject,task)).get_fdata()*th_mat
        ecc_th_mat = nb.load('{}/{}_task-{}_par-ecc.nii.gz'.format(fit_dir,subject,task)).get_fdata()*th_mat
        sd_th_mat = nb.load('{}/{}_task-{}_par-sd.nii.gz'.format(fit_dir,subject,task)).get_fdata()*th_mat
        x_th_mat = nb.load('{}/{}_task-{}_par-x.nii.gz'.format(fit_dir,subject,task)).get_fdata()*th_mat
        y_th_mat = nb.load('{}/{}_task-{}_par-y.nii.gz'.format(fit_dir,subject,task)).get_fdata()*th_mat
        amp_th_mat = nb.load('{}/{}_task-{}_par-amplitude.nii.gz'.format(fit_dir,subject,task)).get_fdata()*th_mat
        bsl_th_mat = nb.load('{}/{}_task-{}_par-baseline.nii.gz'.format(fit_dir,subject,task)).get_fdata()*th_mat
        
        # creat tsv
        for roi_num, roi in enumerate(rois):
            # load roi
            lh_mat = nb.load("{}/{}_{}_L.nii.gz".format(mask_dir, roi, cortical_mask)).get_fdata()
            rh_mat = nb.load("{}/{}_{}_R.nii.gz".format(mask_dir, roi, cortical_mask)).get_fdata()
            roi_mat = lh_mat + rh_mat
            roi_mat[roi_mat==0] = np.nan

            # select data by roi mask
            r2_roi_th_mat = r2_th_mat[roi_mat==True]
            ecc_roi_th_mat = ecc_th_mat[roi_mat==True]
            sd_roi_th_mat = sd_th_mat[roi_mat==True]
            x_roi_th_mat = x_th_mat[roi_mat==True]
            y_roi_th_mat = y_th_mat[roi_mat==True]
            amp_roi_th_mat = amp_th_mat[roi_mat==True]
            bsl_roi_th_mat = bsl_th_mat[roi_mat==True]
            
            # create dataframe
            df_roi = pd.DataFrame({'subject': [subject] * r2_roi_th_mat.shape[0],
                                   'roi': [roi] * r2_roi_th_mat.shape[0],
                                   'r2': r2_roi_th_mat,
                                   'ecc': ecc_roi_th_mat,
                                   'sd': sd_roi_th_mat,
                                   'x': x_roi_th_mat,
                                   'y': y_roi_th_mat,
                                   'amplitude': amp_roi_th_mat,
                                   'baseline': bsl_roi_th_mat})

            # rank based on r2
            if task == 'FullScreen': 
                df_roi['rank_fs_r2']=df_roi.groupby('roi')['r2'].rank(method='max',ascending=False)
            else:
                df_fs = pd.read_csv("{}/{}_task-FullScreen_prf_threshold_par.tsv".format(tsv_dir,subject,task),sep="\t")
                df_roi['rank_fs_r2'] = np.array(df_fs.loc[(df_fs.roi == roi)]['rank_fs_r2'])
                
            # get best voxels
            df_best_roi = df_roi[(df_roi.rank_fs_r2<=best_voxels_num)]
            
            # across roi
            if roi_num > 0: 
                df = pd.concat([df,df_roi], ignore_index=True)
                df_best = pd.concat([df_best,df_best_roi], ignore_index=True)                
            else:
                df = df_roi
                df_best = df_best_roi
        
        # save dataframe
        df_fn = "{}/{}_task-{}_prf_threshold_par.tsv".format(tsv_dir,subject,task)
        print('saving {}'.format(df_fn))
        df.to_csv(df_fn, sep="\t", na_rep='NaN',index=False)
        
        df_best_fn = "{}/{}_task-{}_prf_threshold_par_best{}.tsv".format(tsv_dir,subject,task,int(best_voxels_num))
        print('saving {}'.format(df_best_fn))
        df_best.to_csv(df_best_fn, sep="\t", na_rep='NaN',index=False)
        
        # across subject
        if subject_num == 0: df_group = df
        else: df_group = pd.concat([df_group, df])
        
        if subject_num == 0: df_best_group = df_best
        else: df_best_group = pd.concat([df_best_group, df_best])
        
    # save group data
    df_group_fn = "{}/group_task-{}_prf_threshold_par.tsv".format(group_tsv_dir,task)
    print('saving {}'.format(df_group_fn))
    df_group.to_csv(df_group_fn, sep="\t", na_rep='NaN')
    
    df_best_group_fn = "{}/group_task-{}_prf_threshold_par_best{}.tsv".format(group_tsv_dir,task,int(best_voxels_num))
    print('saving {}'.format(df_best_group_fn))
    df_best_group.to_csv(df_best_group_fn, sep="\t", na_rep='NaN')

saving /home/mszinte/disks/meso_S/data/gaze_prf/derivatives/pp_data/sub-001/prf/tsv/sub-001_task-FullScreen_prf_threshold_par.tsv
saving /home/mszinte/disks/meso_S/data/gaze_prf/derivatives/pp_data/sub-001/prf/tsv/sub-001_task-FullScreen_prf_threshold_par_best250.tsv
saving /home/mszinte/disks/meso_S/data/gaze_prf/derivatives/pp_data/sub-002/prf/tsv/sub-002_task-FullScreen_prf_threshold_par.tsv
saving /home/mszinte/disks/meso_S/data/gaze_prf/derivatives/pp_data/sub-002/prf/tsv/sub-002_task-FullScreen_prf_threshold_par_best250.tsv
saving /home/mszinte/disks/meso_S/data/gaze_prf/derivatives/pp_data/sub-003/prf/tsv/sub-003_task-FullScreen_prf_threshold_par.tsv
saving /home/mszinte/disks/meso_S/data/gaze_prf/derivatives/pp_data/sub-003/prf/tsv/sub-003_task-FullScreen_prf_threshold_par_best250.tsv
saving /home/mszinte/disks/meso_S/data/gaze_prf/derivatives/pp_data/sub-004/prf/tsv/sub-004_task-FullScreen_prf_threshold_par.tsv
saving /home/mszinte/disks/meso_S/data/gaze_prf/derivatives/pp_dat

In [3]:
# Compute permutation statistics results
for type_analysis in type_analyses:
    # load data across participants
    ab_df = pd.read_csv("{}/group_task-FullScreenAttendBar_prf_threshold_par{}.tsv".format(group_tsv_dir,type_analysis),sep="\t")
    af_df = pd.read_csv("{}/group_task-FullScreenAttendFix_prf_threshold_par{}.tsv".format(group_tsv_dir,type_analysis),sep="\t")
    r2_ab_df = ab_df.groupby(['roi','subject']).r2.mean().reset_index(name='r2')
    r2_af_df = af_df.groupby(['roi','subject']).r2.mean().reset_index(name='r2')

    for roi_num, roi in enumerate(rois):

        ab_r2_array = np.array(r2_ab_df[r2_af_df.roi==roi].r2)
        af_r2_array = np.array(r2_af_df[r2_af_df.roi==roi].r2)
        perm_res = permutation_test((ab_r2_array, af_r2_array), statistic, n_resamples=n_permutation, alternative='two-sided',
                                    permutation_type='samples', vectorized=True, axis=0)

        p_val_unilateral = perm_res.pvalue/2
        p_val_bilateral = perm_res.pvalue

        if p_val_unilateral > 0.05: p_text_unilateral = 'p = {:1.2f}'.format(p_val_unilateral)
        if p_val_unilateral < 0.05: p_text_unilateral = 'p < 0.05'
        if p_val_unilateral < 0.01: p_text_unilateral = 'p < 0.01'
        if p_val_unilateral < 0.001: p_text_unilateral = 'p < 0.001'
        if p_val_unilateral < 0.0001: p_text_unilateral = 'p < 0.0001'

        if p_val_bilateral > 0.05: p_text_bilateral = 'p = {:1.2f}'.format(p_val_bilateral)
        if p_val_bilateral < 0.05: p_text_bilateral = 'p < 0.05'
        if p_val_bilateral < 0.01: p_text_bilateral = 'p < 0.01'
        if p_val_bilateral < 0.001: p_text_bilateral = 'p < 0.001'
        if p_val_bilateral < 0.0001: p_text_bilateral = 'p < 0.0001'

        df_stats = pd.DataFrame({  'roi': [roi],
                                   'ab_r2_mean': np.nanmean(ab_r2_array),
                                   'ab_r2_sem': np.nanstd(ab_r2_array)/np.sqrt(ab_r2_array.shape[0]-1),
                                   'af_r2_mean': np.nanmean(af_r2_array),
                                   'af_r2_sem': np.nanstd(af_r2_array)/np.sqrt(af_r2_array.shape[0]-1),
                                   'ab_af_diff': perm_res.statistic,
                                   'p_val_unilateral': p_val_unilateral,
                                   'p_text_unilateral': p_text_unilateral,
                                   'p_val_bilateral': p_val_bilateral,
                                   'p_text_bilateral': p_text_bilateral
                                  })

        # across subject
        if roi_num == 0: df_stats_roi = df_stats
        else: df_stats_roi = pd.concat([df_stats_roi, df_stats])
    df_stats_roi_fn = "{}/group_task-FullScreen_prf_threshold_stats{}.tsv".format(group_tsv_dir,type_analysis)
    print('saving {}'.format(df_stats_roi_fn))
    df_stats_roi.to_csv(df_stats_roi_fn, sep="\t", na_rep='NaN')


saving /home/mszinte/disks/meso_S/data/gaze_prf/derivatives/pp_data/group/prf/tsv/group_task-FullScreen_prf_threshold_stats.tsv
saving /home/mszinte/disks/meso_S/data/gaze_prf/derivatives/pp_data/group/prf/tsv/group_task-FullScreen_prf_threshold_stats_best250.tsv


#### Eccentricity effect

In [4]:
# Compute permutation tests
tsv_dir = '{}/group/prf/tsv'.format(pp_dir, subject)
for type_analysis in type_analyses:
    af_df = pd.read_csv("{}/group_task-FullScreenAttendFix_prf_threshold_par{}.tsv".format(tsv_dir,type_analysis), sep="\t")
    ab_df = pd.read_csv("{}/group_task-FullScreenAttendBar_prf_threshold_par{}.tsv".format(tsv_dir,type_analysis), sep="\t")
    af_df.roi = pd.Categorical(af_df.roi,categories=rois)
    ab_df.roi = pd.Categorical(af_df.roi,categories=rois)

    for roi_num, roi in enumerate(rois):
        ab_df_binned = ab_df[ab_df.roi==roi].assign(ecc_bin=pd.qcut(ab_df.ecc, n_ecc_bins))
        af_df_binned = af_df[af_df.roi==roi].assign(ecc_bin=pd.qcut(af_df.ecc, n_ecc_bins))

        ab_df_binned_mean = ab_df_binned.groupby(['subject','ecc_bin']).r2.mean().reset_index(name='r2_mean').assign(ecc_bin_num=np.concatenate([np.arange(0,n_ecc_bins)]*8))
        af_df_binned_mean = af_df_binned.groupby(['subject','ecc_bin']).r2.mean().reset_index(name='r2_mean').assign(ecc_bin_num=np.concatenate([np.arange(0,n_ecc_bins)]*8))

        for ecc_bin_num in np.arange(0,n_ecc_bins):
            ab_df_binned_r2_array = np.array(ab_df_binned_mean[ab_df_binned_mean.ecc_bin_num==ecc_bin_num].r2_mean)
            af_df_binned_r2_array = np.array(af_df_binned_mean[af_df_binned_mean.ecc_bin_num==ecc_bin_num].r2_mean)

            perm_res = permutation_test((ab_df_binned_r2_array, af_df_binned_r2_array), statistic, n_resamples=n_permutation, alternative='two-sided',
                                         permutation_type='samples', vectorized=True, axis=0)

            p_val_unilateral = perm_res.pvalue/2
            p_val_bilateral = perm_res.pvalue

            if p_val_unilateral > 0.05: p_text_unilateral = 'p = {:1.2f}'.format(p_val_unilateral)
            if p_val_unilateral < 0.05: p_text_unilateral = 'p < 0.05'
            if p_val_unilateral < 0.01: p_text_unilateral = 'p < 0.01'
            if p_val_unilateral < 0.001: p_text_unilateral = 'p < 0.001'
            if p_val_unilateral < 0.0001: p_text_unilateral = 'p < 0.0001'

            if p_val_bilateral > 0.05: p_text_bilateral = 'p = {:1.2f}'.format(p_val_bilateral)
            if p_val_bilateral < 0.05: p_text_bilateral = 'p < 0.05'
            if p_val_bilateral < 0.01: p_text_bilateral = 'p < 0.01'
            if p_val_bilateral < 0.001: p_text_bilateral = 'p < 0.001'
            if p_val_bilateral < 0.0001: p_text_bilateral = 'p < 0.0001'

            df_stats = pd.DataFrame({  'roi': [roi],
                                       'ecc_bin_num': [ecc_bin_num],
                                       'ab_r2_mean': np.nanmean(ab_df_binned_r2_array),
                                       'ab_r2_sem': np.nanstd(ab_df_binned_r2_array)/np.sqrt(ab_df_binned_r2_array.shape[0]-1),
                                       'af_r2_mean': np.nanmean(af_df_binned_r2_array),
                                       'af_r2_sem': np.nanstd(af_df_binned_r2_array)/np.sqrt(af_df_binned_r2_array.shape[0]-1),
                                       'ab_af_diff': perm_res.statistic,
                                       'p_val_unilateral': p_val_unilateral,
                                       'p_text_unilateral': p_text_unilateral,
                                       'p_val_bilateral': p_val_bilateral,
                                       'p_text_bilateral': p_text_bilateral,
                                      })
            # across eccentricity
            if ecc_bin_num == 0: df_stats_ecc_bin_num = df_stats
            else: df_stats_ecc_bin_num = pd.concat([df_stats_ecc_bin_num, df_stats])

        # across roi
        if roi_num == 0: df_stats_ecc_bin_num_roi = df_stats_ecc_bin_num
        else: df_stats_ecc_bin_num_roi = pd.concat([df_stats_ecc_bin_num_roi, df_stats_ecc_bin_num])    
    
    df_stats_ecc_bin_num_roi_fn = "{}/group_task-FullScreen_prf_threshold_ecc_bin_num_stats{}.tsv".format(group_tsv_dir,type_analysis)
    print('saving {}'.format(df_stats_ecc_bin_num_roi_fn))
    df_stats_ecc_bin_num_roi.to_csv(df_stats_ecc_bin_num_roi_fn, sep="\t", na_rep='NaN')

saving /home/mszinte/disks/meso_S/data/gaze_prf/derivatives/pp_data/group/prf/tsv/group_task-FullScreen_prf_threshold_ecc_bin_num_stats.tsv
saving /home/mszinte/disks/meso_S/data/gaze_prf/derivatives/pp_data/group/prf/tsv/group_task-FullScreen_prf_threshold_ecc_bin_num_stats_best250.tsv
